## Arduino Data Logging in Python

In [3]:
import serial
import serial.tools.list_ports
import csv
import datetime
import geocoder
g = geocoder.ip('me')
latitude = g.latlng[0]
longitude = g.latlng[1]
print(latitude, longitude)


43.4715 -80.5454


2023-07-09 14:36:43.791468


In [5]:
run sun.py

In [80]:
#Functions from A1

def I_t(el):
    # negative elevation not meaningful
    el2 = el.copy()
    el2[el2<0] = 0
    AM = 1/np.cos(np.radians(90-el2))
    I = 1.1*(1353*0.7**AM**0.678)
    return I

def current(el, area):
    a_m2 = area/10000
    power = I_t(el)*a_m2*0.2 # energy/area * area * efficiency
    current = power/5 #volts
    return current*1000

def R_opt(beta_ax, az_ax, el, az,limit=90):
    beta_ax = np.radians(beta_ax)
    az_ax = np.radians(az_ax)
    
    zen = np.radians(90 - el)
    
    azrad = np.radians(az)
        
    arg = np.sin(zen)*np.sin(azrad-az_ax)/ \
            (np.sin(zen)*np.cos(azrad-az_ax)*np.sin(beta_ax) \
             + np.cos(zen)*np.cos(beta_ax))
    
    phi = np.where((arg < 0) & ((azrad-az_ax) > 0) , 180, 
            np.where((arg > 0) & ((azrad-az_ax) < 0), -180,0))
    
    
    R = np.degrees(np.arctan(arg)) + phi
    
    R[R>90] = limit
    R[R<-90] = -limit
    
    return R

def inc_SA(R, beta_ax, az_ax, el, az): 
    assert R.shape[0] == el.shape[0]
    R = np.radians(R)
    beta_ax = np.radians(beta_ax)
    az_ax = np.radians(az_ax)
    el = np.radians(90-el)
    az = np.radians(az)
    
    arg = np.cos(R)*(np.sin(el)*np.cos(az-az_ax)*np.sin(beta_ax) \
                    +np.cos(el)*np.cos(beta_ax)) + \
                    np.sin(R)*np.sin(el)*np.sin(az-az_ax)
    return np.degrees(np.arccos(arg))

def power(el, az, area, period, motor_power, ret_error = False):

    beta_ax = 20
    az_ax = 180

    # time vector 
    time = np.repeat(np.arange(0,24),60) + np.tile(np.arange(0,60),24)*1/60
    
    # daylight data only
    time = time[el>0]
    az = az[el > 0]
    el = el[el > 0]

    # create the discrete time vector with spacing of period (mins)
    # the vector is centred around solar noon (middle of daylight)
    tx1 = np.arange(time[int(time.shape[0]/2)],0, -period/60)
    tx2 = np.arange(time[int(time.shape[0]/2)],time[-1], period/60)
    tx = np.append(np.flip(tx1[1:]),tx2)
    
    # obtain indices in same way
    idx1 = np.arange(int(time.shape[0]/2),0, -period)
    idx2 = np.arange(int(time.shape[0]/2),time.shape[0], period)

    #Ewe will use this index vector to discretize the Rotation array
    idx1 = idx1[1:].repeat(period)[:int(time.shape[0]/2 - period)]
    idx2 = idx2.repeat(period)[:int(time.shape[0]/2)]
    idx = np.append(np.flip(idx1),idx2)
    
    # might need to pad the beginning and the end so idx is the correct size
    pad = (time.shape[0] - idx.shape[0])/2
    if pad > 0:
        idx = np.pad(idx,int(pad),mode = 'edge')
    if pad%2 > 0:
        idx = np.append(idx,idx[-1])

    R = R_opt(beta_ax,az_ax,el,az)
    
    # down-sample the rotation angle by the period
    Rd = R[idx][:el.shape[0]]

    # need power generation over the day
    # discreted adjustment
    inc = inc_SA(Rd, beta_ax, az_ax, el, az)
    # continuous adjustment
    inc_ideal = inc_SA(R, beta_ax, az_ax, el, az)

    #calculate loss due to misalignment
    cos_inc = np.cos(np.radians(inc))
    cos_inc_ideal = np.cos(np.radians(inc_ideal))

    # anything less than zero not possible  
    cos_inc[cos_inc<0] = 0

    # power: (current (mA) * volts * 1000) [Watts] * cos(theta)
    power_ideal = current(el,area)*5/1000
    power = power_ideal*cos_inc

    power_harvested = np.trapz(power,dx=1/60) #watt-hours
    
    # number of changes/3600 (hours of movement) * 1.5 watts
    power_expended = tx.shape[0]/3600*1.5 #watt hours 
    
    # objective function
    total = power_harvested - power_expended #watt hours 

    #if we want the data as a function of time (specified in input)
    if ret_error:
        # Loss of power over time due to discrete adjustment interval 
        error = power_ideal*cos_inc_ideal - power
        return error, power_ideal, power, Rd
    else:
        return total

In [84]:
today = datetime.datetime.now()
print(datetime.datetime.now())

# B axis calculation
if today.month in [10, 11,12,1,2, 3]: #in winter months, latitude + 15
    tilt_angle = latitude + 15
elif today.month in [4,5,6, 7, 8, 9]: #in summer months, latitude - 15
    tilt_angle = latitude - 15


#Initial azimuth calculation 
time_zone_shift = 4


# def sunPosition(year, month, day, hour=12, m=0, s=0,lat=43.5, long=-80.5):
initial_angle, el = sunPosition(year =2023, month = today.month, day = today.day, hour=today.hour+time_zone_shift, lat=latitude, long=longitude)

if initial_angle < 90:
    initial_angle = 90
if initial_angle > 270: 
    initial_angle = 270 

initial_rotation_ang = 180 - initial_angle 

# if rotation_ang < 0: 
#     #rotate counter clockwise
# if rotation_ang > 0: 
#     #rotate clockwise 


#Time and amount of rotation calculation 
period= np.arange(1,200)#mins
m_power = 1.2 # Watts (0.240mAh * 5V) 
area = 35 # cm^2  (5cm x 7cm)
p = np.array([power(Wjun[:,1],Wjun[:,0],area,x,m_power) for x in period])
print('Ideal interval of adjustment is ' + str(period[p.argmax()]) + ' mins')

optimal_adjustment_mins = 41;
number_of_daylight_hours = 15;

time_zone = 4
hrs = np.arange(0+time_zone,24+time_zone)
mins = np.arange(0,60)
Wjun = np.array([sunPosition(year=today.year,month=today.month, day=today.day,hour=hr,m=mn, lat=latitude, long=longitude ) 
                for hr,mn in zip(np.repeat(hrs,60),np.tile(mins,24))])
el = Wjun[:,1]
daylight_hours = el[el>0].shape[0]/60
adjustment_amount = 1024/((number_of_daylight_hours*60)/optimal_adjustment_mins);

print(adjustment_amount)



#SEND adjusment_amount, tilt_angle AND initial_rotation_ang TO ARDUINO 

2023-07-09 17:54:31.551222
Ideal interval of adjustment is 41 mins
46.648888888888884


### Find port

In [10]:
ports = serial.tools.list_ports.comports()

for i,port in enumerate(ports): 
    print(str(i) + ': ' + port.device)

0: COM5
1: COM6


### Select Port

In [11]:
# Change "4" to ESP32 bluetooth port
port = ports[1].device

comm_rate = 115200 #baud
fname = "data.csv" #location

#Begin communication
ser = serial.Serial(port, comm_rate)

SerialException: could not open port 'COM6': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

### (Optional) Send message to MCU to initiate operation

In [9]:
ser.write(bytes('hello', 'utf-8'))

NameError: name 'ser' is not defined

### Receive communication
Loop this to receive ongoing messages, modify to append data to file, or to an array, etc. 

In [ ]:
file = open(fname, "a")
getData=ser.readline()
dataString = getData.decode('utf-8')[:-2]